In [213]:
import os
# CUR_PATH = os.path.dirname(os.path.realpath(__file__))

from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE

PPT_PATH = "sample_template.pptx"
prs = Presentation(PPT_PATH)

In [ ]:
import re
from copy import deepcopy
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE

def extract_subkeys(header: str) -> list[str]:
    """
    셀 placeholder 텍스트를 'a-b', 'c(d)', 단일 키 형태로 해석하여
    실제 키 이름 목록을 반환합니다.
    """
    header = header.strip()
    if "-" in header:
        return [h.strip() for h in header.split("-")]
    if "(" in header and header.endswith(")"):
        main, rest = header.split("(", 1)
        return [main.strip(), rest[:-1].strip()]
    return [header]

def flatten_replace_dict(d: dict) -> dict:
    """
    중첩된 dict / list-of-dict 구조를 순회하며
    최종 scalar 값을 키→값 매핑으로 평탄화합니다.
    (list-of-dict는 첫 요소만 사용)
    """
    flat: dict = {}
    def recurse(x):
        if isinstance(x, dict):
            for k, v in x.items():
                if isinstance(v, (str, bool, int, float)):
                    flat[k] = v
                elif isinstance(v, dict):
                    recurse(v)
                elif isinstance(v, list) and v and isinstance(v[0], dict):
                    recurse(v[0])
        elif isinstance(x, list) and x and isinstance(x[0], dict):
            recurse(x[0])
    recurse(d)
    return flat

def apply_text_replacements(tf, replace_dict: dict):
    """
    텍스트 프레임 내 run 단위로, 키를 길이 내림차순으로
    whole-word lookaround 치환합니다.
    """
    flat = flatten_replace_dict(replace_dict)
    # 키를 길이 내림차순 정렬 → 'project_name' 먼저, 'name' 나중
    keys = sorted(flat.keys(), key=len, reverse=True)
    # 각 키마다 단어 경계 lookaround 패턴
    patterns = {
        key: re.compile(rf"(?<![A-Za-z_]){re.escape(key)}(?![A-Za-z_])")
        for key in keys
    }

    for para in tf.paragraphs:
        for run in para.runs:
            text = run.text
            for key in keys:
                text = patterns[key].sub(str(flat[key]), text)
            run.text = text

def apply_replacements(shape, replace_dict: dict):
    """
    하나의 Shape에 대해:
      1) 테이블 처리: placeholder row 복제 → Run.text 교체 (서식 보존)
      2) 텍스트 프레임 처리: apply_text_replacements 호출
    """
    # 1) 테이블 처리
    if hasattr(shape, "has_table") and shape.has_table:
        table = shape.table
        rows = table.rows
        if not rows:
            return

        # 첫 행을 placeholder row로 간주
        placeholder_row = rows[0]
        placeholder_texts = [cell.text.strip() for cell in placeholder_row.cells]
        placeholder_subkeys = {
            sub for txt in placeholder_texts
                for sub in extract_subkeys(txt)
        }

        # 후보: top-level list-of-dict + nested list-of-dict
        candidates = []
        for v in replace_dict.values():
            if isinstance(v, list) and v and isinstance(v[0], dict):
                candidates.append(v)
                for item in v:
                    for subv in item.values():
                        if isinstance(subv, list) and subv and isinstance(subv[0], dict):
                            candidates.append(subv)

        # 중복 제거, 순서 보존
        uniq_cands = []
        seen = set()
        for lst in candidates:
            id0 = id(lst)
            if id0 not in seen:
                uniq_cands.append(lst)
                seen.add(id0)

        # placeholder_subkeys ⊆ item.keys() 인 리스트 찾기
        for cand_list in uniq_cands:
            item_keys = set(cand_list[0].keys())
            if placeholder_subkeys.issubset(item_keys):
                n = len(cand_list)
                tbl = table._tbl            # xml element
                tr = placeholder_row._tr    # xml row

                # placeholder row 복제 (n-1)번
                for _ in range(n - 1):
                    tbl.append(deepcopy(tr))

                # 각 행의 각 셀 Run.text만 교체 (서식 보존)
                for i, item in enumerate(cand_list):
                    row = table.rows[i]
                    for j, cell in enumerate(row.cells):
                        header = placeholder_texts[j]
                        tmpl = re.sub(r"([A-Za-z_]+)", r"{\1}", header)
                        try:
                            new_text = tmpl.format_map(item)
                        except KeyError:
                            new_text = ""
                        # placeholder_text가 들어있던 첫 번째 Run만 바꿔 줌
                        for para in cell.text_frame.paragraphs:
                            if para.runs:
                                para.runs[0].text = new_text
                            else:
                                run = para.add_run()
                                run.text = new_text
                            break
                return  # 테이블 처리 후 바로 종료

    # 2) 텍스트 프레임 처리
    if hasattr(shape, "has_text_frame") and shape.has_text_frame:
        apply_text_replacements(shape.text_frame, replace_dict)

def replace_in_presentation(prs: Presentation, replace_dict: dict):
    """
    Presentation 전체에 대해 GROUP/일반 Shape 모두 처리
    """
    for slide in prs.slides:
        for shape in slide.shapes:
            if shape.shape_type == MSO_SHAPE_TYPE.GROUP:
                for sub in shape.shapes:
                    apply_replacements(sub, replace_dict)
            else:
                apply_replacements(shape, replace_dict)

In [215]:
import ast

ppt_str = """
{'name': '권기범', 'email': 'qja1998@naver.com', 'phone_number': '010-0000-0000', 'github_url': 'https://github.com/qja1998', 'blog_url': 'qja1998.github.io', 'enter_slogan': '데이터 기반 문제 해결과 효율적인 AI 시스템 구현에 집중하는 개발자, 권기범입니다.', 'certificate_list': [{'certificate_name': '정보처리기사', 'certificate_authority': '근로진흥원', 'certificate_date': '2023-11-13'}], 'language_list': [{'language_name': '토익스피킹', 'language_authority': 'YBM', 'language_date': '2024-03-13', 'language_grade': 'IM3'}], 'tech_stack_list': [{'stack_name': 'Python', 'stack_level': '상'}, {'stack_name': 'Machine Learning', 'stack_level': '중'}, {'stack_name': 'Git', 'stack_level': '중'}], 'education_list': [{'education_name': '경상대학교', 'major': '항공소프트웨어', 'entrance_date': '2017-03-02', 'graduate_date': '2024-02-16', 'transfered': False, 'type': 3}], 'award_list': [{'award_name': '캡스톤 경진대회', 'award_authority': 'LINC 사업단', 'award_date': '2024-10-13', 'award_details': '금상'}], 'activity_list': [{'activity_name': 'SSAFY', 'activity_authority': '삼성', 'activity_start_date': '2025-03-13', 'activity_end_date': '2025-05-13', 'activity_details': 'Data track'}], 'experience_list': [{'experience_name': 'co2-emission-management', 'experience_start_date': '2023.06', 'experience_end_date': 'Present', 'experience_role': '주요 기능(배출권 값 예측 및 데이터 처리) 개발자 및 프로젝트 문서화 담당'}], 'project_details': [{'project_name': 'CO2 Emission Management', 'project_start_date': '2023.06', 'project_end_date': '2024.06', 'user_roles': ['백엔드 개발', 'AI 모델 구현', '데이터 처리 로직 개발'], 'project_description': 'AI 기반 탄소 배출량 관리 시스템으로, 배출권 값 예측 및 관리 기능을 제공합니다. 주요 기능으로는 배출권 값 예측, 데이터 처리, 불필요한 데이터 정제 등이 포함되어 있습니다.', 'repo_name': 'co2-emission-management', 'troubleshootings': [{'troubleshooting_name': '배출권 값 예측 기능 구현', 'troubleshooting_detail': '탄소 배출권 값을 예측하는 기능이 필요했음.', 'troubleshooting_solve': 'prediction 구현 완료 (커밋 #94fb0d9)로 AI 기반 예측 로직을 개발하여 문제를 해 결함.'}, {'troubleshooting_name': '불필요한 데이터 처리', 'troubleshooting_detail': '탄소 배출권 관련 데이터를 받아오는 과정에서 불필요한 부분이 존재함.', 'troubleshooting_solve': '탄소 배출권 수정 받아오는 파일에서 불필요한 부분 제거 (커밋 #fd7e4c4)로 데이터 정제 수행.'}], 'tech_section': [{'tech_domain': 'Backend', 'tech_name': 'Python (추정)'}, {'tech_domain': 'AI/ML', 'tech_name': 'Prediction Model (커밋 메시지 기반 추론)'}], 'project_contributions': ['prediction 기능 구현 완료 ( 커밋 #94fb0d9)', '배출권 값 가져오는 기능 구현 (커밋 #96b12f3)', '탄소 배출권 데이터 정제 및 불필요한 부분 제거 (커밋 #fd7e4c4)']}], 'final_slogan': '데이터와 AI  기술을 바탕으로 실제 문제를 해결하며, 꾸준한 개선과 성장으로 미래를 만들어가는 개발자 권기범(qja1998)입니다.'}
"""
ppt_dict = ast.literal_eval(ppt_str)
replace_in_presentation(prs, ppt_dict)

In [216]:
ppt_dict

{'name': '권기범',
 'email': 'qja1998@naver.com',
 'phone_number': '010-0000-0000',
 'github_url': 'https://github.com/qja1998',
 'blog_url': 'qja1998.github.io',
 'enter_slogan': '데이터 기반 문제 해결과 효율적인 AI 시스템 구현에 집중하는 개발자, 권기범입니다.',
 'certificate_list': [{'certificate_name': '정보처리기사',
   'certificate_authority': '근로진흥원',
   'certificate_date': '2023-11-13'}],
 'language_list': [{'language_name': '토익스피킹',
   'language_authority': 'YBM',
   'language_date': '2024-03-13',
   'language_grade': 'IM3'}],
 'tech_stack_list': [{'stack_name': 'Python', 'stack_level': '상'},
  {'stack_name': 'Machine Learning', 'stack_level': '중'},
  {'stack_name': 'Git', 'stack_level': '중'}],
 'education_list': [{'education_name': '경상대학교',
   'major': '항공소프트웨어',
   'entrance_date': '2017-03-02',
   'graduate_date': '2024-02-16',
   'transfered': False,
   'type': 3}],
 'award_list': [{'award_name': '캡스톤 경진대회',
   'award_authority': 'LINC 사업단',
   'award_date': '2024-10-13',
   'award_details': '금상'}],
 'activity_list

In [217]:
prs.save('./test.pptx')